In [4]:
from os import listdir
from os.path import isfile, join

mypath = ".//dataset4//images//"


file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]

print(str(len(file_names)) + ' images loaded')

642 images loaded


In [5]:
import cv2
import numpy as np
import sys
import os
import shutil

In [6]:
def make_dir(directory):
        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.makedirs(directory)

In [7]:
green_count = 0
red_count = 0
others_count=0
training_size = 100
test_size =80
training_images = []
training_labels = []
test_images = []
test_labels = []
size=1
green_dir_train = "./datasets4/images/train/green/"
red_dir_train = "./datasets4/images/train/red/"
others_dir_train = "./datasets4/images/train/others/"
green_dir_val = "./datasets4/images/validation/green/"
red_dir_val = "./datasets4/images/validation/red/"
others_dir_val = "./datasets4/images/validation/others/"



make_dir(green_dir_train)
make_dir(red_dir_train)
make_dir(others_dir_train)
make_dir(green_dir_val)
make_dir(red_dir_val)
make_dir(others_dir_val)

def getZeros(number): #for counting images
    if(number > 10 and number < 100):
        return "0"
    if(number < 10):
        return "00"
    else:
        return ""

for i, file in enumerate(file_names):
    
    if file_names[i][0] == "g":
        if green_count>=training_size+test_size:
            continue
        else:
            #print(file,end=',')
            image = cv2.imread(mypath+file)
            image = cv2.resize(image,(size, size), interpolation = cv2.INTER_AREA)
            if green_count < training_size:
                training_images.append(image)
                training_labels.append(0)
                zeros = getZeros(green_count)
                cv2.imwrite(green_dir_train + "green" + str(zeros) + str(green_count) + ".jpg", image)
            if green_count >= training_size and green_count <= training_size+test_size:
                test_images.append(image)
                test_labels.append(0)
                zeros = getZeros(green_count-1000)
                cv2.imwrite(green_dir_val + "green" + str(zeros) + str(green_count-1000) + ".jpg", image)
        green_count += 1
        
    if file_names[i][0] == "r":
        if red_count >= training_size+test_size:
            continue
        else:
            #print(file,end=',')
            image = cv2.imread(mypath+file)
            image = cv2.resize(image, (size,size), interpolation = cv2.INTER_AREA)
            if red_count < training_size:
                training_images.append(image)
                training_labels.append(1)
                zeros = getZeros(red_count)
                cv2.imwrite(red_dir_train + "red" + str(zeros) + str(red_count) + ".jpg", image)
            if red_count >= training_size and red_count <= training_size+test_size:
                test_images.append(image)
                test_labels.append(1)
                zeros = getZeros(red_count-1000)
                cv2.imwrite(red_dir_val + "red" + str(zeros) + str(red_count-1000) + ".jpg", image)
        red_count += 1
        
   


    if file_names[i][0] == "o":
        if others_count>=training_size+test_size:
            continue
        else:
            #print(file,end=',')
            image = cv2.imread(mypath+file)
            image = cv2.resize(image,(size, size), interpolation = cv2.INTER_AREA)
            if others_count < training_size:
                training_images.append(image)
                training_labels.append(2)
                zeros = getZeros(others_count)
                cv2.imwrite(others_dir_train + "others" + str(zeros) + str(others_count) + ".jpg", image)
            if others_count >= training_size and others_count <= training_size+test_size:
                test_images.append(image)
                test_labels.append(2)
                zeros = getZeros(others_count-1000)
                cv2.imwrite(others_dir_val + "others" + str(zeros) + str(others_count-1000) + ".jpg", image)
        others_count += 1
        
    
        
        
    if green_count == training_size+test_size+1 and red_count == training_size+test_size+1 and others_count==training_size+test_size+1:
        break

print("Training and Test Data Extraction Complete")

Training and Test Data Extraction Complete


In [8]:
np.savez('colour_blind_training_data.npz', np.array(training_images))
np.savez('colour_blind_training_labels.npz', np.array(training_labels))
np.savez('colour_blind_test_data.npz', np.array(test_images))
np.savez('colour_blind_test_labels.npz', np.array(test_labels))

In [9]:
import numpy as np

def load_data_training_and_test(datasetname):
    
    npzfile = np.load(datasetname + "_training_data.npz")
    train = npzfile['arr_0']
    
    npzfile = np.load(datasetname + "_training_labels.npz")
    train_labels = npzfile['arr_0']
    
    npzfile = np.load(datasetname + "_test_data.npz")
    test = npzfile['arr_0']
    
    npzfile = np.load(datasetname + "_test_labels.npz")
    test_labels = npzfile['arr_0']

    return (train, train_labels), (test, test_labels)

for i in range(1,11):
    random = np.random.randint(0, len(training_images))
    cv2.imshow("image_"+str(i), training_images[random])
    if training_labels[random] == 0:
        print(str(i) + " - green")
    if training_labels[random] == 1:
        print(str(i) + " - red")
    else:
        print(str(i) + " - others")
    
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

In [11]:
from keras.utils import np_utils

In [12]:
(x_train, y_train), (x_test, y_test) = load_data_training_and_test("colour_blind")

# Reshaping our label data from (2000,) to (2000,1) and test data from (1000,) to (1000,1)
#y_train = y_train.reshape(y_train.shape[0], 1)
#y_test = y_test.reshape(y_test.shape[0], 1)
y_train=np_utils.to_categorical(y_train)
y_test=np_utils.to_categorical(y_test)
# Change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(300, 1, 1, 3)
(300, 3)
(240, 1, 1, 3)
(240, 3)


In [13]:
y_train[2]

array([0., 0., 1.], dtype=float32)

In [14]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


In [15]:
from keras.optimizers import SGD

batch_size = 16
epochs = 15

img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]
input_shape = (img_rows, img_cols, 3)
model=Sequential() #use to make layers
model.add(Conv2D(32,kernel_size=(1,1),activation='relu',input_shape=input_shape))
model.add(Conv2D(64,(1,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=SGD(0.01),metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 1, 32)          128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          2112      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [16]:
history=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test),shuffle=True)
score=model.evaluate(x_test,y_test,verbose=1)
print('test losses',score[0])
print('test accuracy',score[1])

Train on 300 samples, validate on 240 samples
Epoch 1/15
300/300 [==============================] - 1s 4ms/step - loss: 1.1023 - acc: 0.3067 - val_loss: 1.0898 - val_acc: 0.5167
Epoch 2/15
300/300 [==============================] - 0s 178us/step - loss: 1.0952 - acc: 0.3833 - val_loss: 1.0766 - val_acc: 0.7583
Epoch 3/15
300/300 [==============================] - 0s 191us/step - loss: 1.0775 - acc: 0.4800 - val_loss: 1.0646 - val_acc: 0.9500
Epoch 4/15
300/300 [==============================] - 0s 198us/step - loss: 1.0695 - acc: 0.6000 - val_loss: 1.0532 - val_acc: 0.9542
Epoch 5/15
300/300 [==============================] - 0s 194us/step - loss: 1.0573 - acc: 0.6367 - val_loss: 1.0420 - val_acc: 0.9542
Epoch 6/15
300/300 [==============================] - 0s 194us/step - loss: 1.0464 - acc: 0.6867 - val_loss: 1.0302 - val_acc: 0.9542
Epoch 7/15
300/300 [==============================] - 0s 167us/step - loss: 1.0398 - acc: 0.7233 - val_loss: 1.0189 - val_acc: 0.9542
Epoch 8/15
300/300

In [17]:
model.save("colour_blindness_97.h5")

In [18]:
import cv2
import numpy as np
from keras.models import load_model

classifier = load_model('colour_blindness_97.h5')

def draw_test(name, pred, input_im):
    print(pred)
    BLACK = [0,0,0]
    print(type(pred))
    if pred == "0":
        pred = "green"
    if pred == "1":
        pred = "red"
    if pred == "2":
        pred = "others"
    print(pred)
    '''   
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0] ,
                                        cv2.BORDER_CONSTANT,value=BLACK)
    #expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (252, 70) , 
                cv2.FONT_HERSHEY_COMPLEX_SMALL,4, (0,255,0), 2)
    cv2.imshow(name, expanded_image)
    '''

for i in range(0,10):
    rand = np.random.randint(0,len(x_test))
    input_im = x_test[rand]

    imageL = cv2.resize(input_im, None, fx=2, fy=2, 
                        interpolation = cv2.INTER_CUBIC)
    cv2.imshow("Test Image", imageL)

    input_im = input_im.reshape(1,1,1,3) 
    
    ## Get Prediction
    res = str(classifier.predict_classes(input_im, 1, verbose = 0)[0])

    draw_test("Prediction", res,input_im) 
    cv2.waitKey(0)

cv2.destroyAllWindows()




1
<class 'str'>
red
1
<class 'str'>
red
1
<class 'str'>
red
0
<class 'str'>
green
1
<class 'str'>
red
1
<class 'str'>
red
2
<class 'str'>
others
1
<class 'str'>
red
2
<class 'str'>
others
1
<class 'str'>
red


#yeh mera khud ka h...yeh copy ni krna
%matplotlib inline
import matplotlib.pyplot as plt 
nemo = cv2.imread('image.jpg')
plt.imshow(nemo)
plt.show()

In [19]:
#yeh mera khud ka tp h
import cv2
import numpy as np

x = np.random.randint(0,5,(500,500))
img = cv2.imread('image.jpg')
p = img.shape
print(p)
rows,cols,_ = img.shape

for i in range(rows):
    for j in range(cols):
        k = img[i,j]
        #print (k,end=',')

(167, 302, 3)


In [20]:
classifier = load_model('colour_blindness_97.h5')
def new(fr):
    global classifier
    res = str(classifier.predict_classes(fr, 1, verbose = 0)[0])
    #print(res)
    return res

In [25]:
import cv2 

img=cv2.imread('image.jpg')
row=img.shape[0]
col=img.shape[1]
red=0
green=0
others=0
frame1 = img.astype('float32')
frame1 /= 255
for i in range(row):
    for j in range(col):
        a=new(frame1[i][j].reshape(1,1,1,3))
        if(a=='0'):
            red=red+1
        elif(a=='1'):
            green=green+1
        elif(a=='2'):
            others=others+1
print(red)
print(green)
print(others)

9179
11025
30230


In [1]:
import cv2 
import numpy as np
from keras.models import load_model 

classifier=load_model('colour_blindness_97.h5')
def draw_test(name, pred, input_im):
    BLACK = [0,0,0]
    if pred == "[0]":
        pred = "red"
    if pred == "[1]":
        pred = "green"
    if pred == "[2]":
        pred=="others"
    
    
    
    cv2.putText(input_im, str(pred), (180, 70) , 
                cv2.FONT_HERSHEY_COMPLEX_SMALL,2, (0,255,0), 2)
    cv2.imshow(name,input_im)
   
cam = cv2.VideoCapture(0)
while cam.isOpened():
    ret, frame= cam.read(0)
    frames=cv2.resize(frame,(1,1),interpolation=cv2.INTER_AREA)
    img_rows=frames[0].shape[0]
    img_cols=frames[0].shape[1]
    frames=frames.reshape(1,frames.shape[0],img_rows,img_cols)
    frames=frames.astype('float32')
    frames/=255

    print(frames.shape)
    
    pred=str(classifier.predict_classes(frames,verbose=0)[0])
    print(pred)
    draw_test("prediction",pred,frame)
    if cv2.waitKey(1) == 27 :
        break
    
cam.release()
cv2.destroyAllWindows


Using TensorFlow backend.
W0711 21:24:19.239754 139701457000256 deprecation_wrapper.py:119] From /home/adhoc/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 21:24:19.256706 139701457000256 deprecation_wrapper.py:119] From /home/adhoc/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 21:24:19.297136 139701457000256 deprecation_wrapper.py:119] From /home/adhoc/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0711 21:24:19.300342 139701457000256 deprecation_wrapper.py:119] From /home/adhoc/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instea

(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 3)
2
(1, 1, 1, 

<function destroyAllWindows>

In [ ]:
#for trial
import cv2


cap=cv2.VideoCapture(0)

while cap.isOpened():
    status,frame=cap.read()
    hsvimg=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
   
    imgmask=cv2.inRange(hsvimg,(40,50,50),(80,255,255))
   
    greenpart=cv2.bitwise_and(frame,frame,mask=imgmask)
    redpart=cv2.bitwise_and(frame,frame,mask=imgmask)
    cv2.imshow('original',frame)
    
    cv2.imshow('green',greenpart)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()